In [1]:
path = !pwd
path = path[0]

if path != '/scratch/ss15592/NLP-Project':
    %cd '/scratch/ss15592/NLP-Project'

print('Working Directory: ', path)

Working Directory:  /scratch/ss15592/NLP-Project


In [2]:
import sys, os, csv, glob, json, uuid, pickle, math
import gensim, logging
import numpy as np, scipy, pandas as pd
import nltk
nltk.download('punkt')
from operator import itemgetter
import re

''' Dual Embedding Space Testing '''

[nltk_data] Downloading package punkt to /home/ss15592/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


' Dual Embedding Space Testing '

In [3]:
def get_embedding(x, vector_size, model, out=False):
    if x in model.wv and out == True:
        return model.syn1neg[model.wv.key_to_index[x]]
    
    elif x in model.wv and out == False:
        return model.wv[x]
    
    else:
        return np.zeros(vector_size)

In [4]:
query = 'veal'
query_tokens = nltk.word_tokenize(query.lower())
model = gensim.models.Word2Vec.load("./model/w2v-lc.model")

In [5]:
query_embed = [get_embedding(x, model.vector_size, model) for x in query_tokens]
query_len = len(query_embed)

print('Num words in query:', len(query_tokens), 
      '\nNum query word in vectors:', query_len)

Num words in query: 1 
Num query word in vectors: 1


In [6]:
centroid_dict = {}
for fname in glob.iglob('./inputs/centroids/*.p', recursive=False):
    centroid_dict.update(pickle.load(open(fname, "rb")))
    
clean_centroid_dict = {k: centroid_dict[k] for k in centroid_dict 
                       if np.isnan(centroid_dict[k][0]).any()==False}

In [7]:
def score_document(query_embed, doc_centroid):
    cos_similarity = [(1 - scipy.spatial.distance.cosine(qin, doc_centroid)) 
                        for qin in query_embed]
    
    return (sum(cos_similarity)/len(query_embed))

In [8]:
def view_files(doc_list):
    for path in doc_list:
        num = re.findall(r'\d+', path)[0]
        doc_path = './inputs/doc_num/'+ num +'.txt'
        f = open(doc_path, "r", encoding='utf-8')
        print('\033[1m' + 'DOCUMENT NUMBER: ' + '\033[0m', f.read())
        f.close()
        
        doc_path = './inputs/contents/'+ num +'.txt'
        f = open(doc_path, "r", encoding='utf-8')
        print(f.read())
        f.close()
        print()

In [9]:
scores_in_in = []
scores_in_out = []

for k,v in clean_centroid_dict.items():
    scores_in_in.append((k, score_document(query_embed, v[0])))
    scores_in_out.append((k, score_document(query_embed, v[1])))

scores_in_in = sorted(scores_in_in, key=itemgetter(1), reverse=True)
scores_in_out = sorted(scores_in_out, key=itemgetter(1), reverse=True)

In [10]:
top_5_in_in = [x[0] for x in scores_in_in[:5]]
top_5_in_out = [x[0] for x in scores_in_out[:5]]

in_in_top5 = []
in_out_top5 = []

for fname in top_5_in_in:
    in_in_top5.append(fname)
for fname in top_5_in_out:
    in_out_top5.append(fname)

In [11]:
'''IN-IN Results'''
print(scores_in_in[:5])
view_files(in_in_top5)

[('./inputs/tokens/1400.tokens', 0.8747051954269409), ('./inputs/tokens/79.tokens', 0.823441743850708), ('./inputs/tokens/3323.tokens', 0.818074107170105), ('./inputs/tokens/2214.tokens', 0.8144202828407288), ('./inputs/tokens/1662.tokens', 0.8114814758300781)]
DOCUMENT NUMBER:  MED-2468
BACKGROUND AND METHODS: We estimated the prevalence of self-reported asthma in adult Indians and examined several risk factors influencing disease prevalence. Analysis is based on 99 574 women and 56 742 men aged 20–49 years included in India’s third National Family Health Survey, 2005–2006. Multiple logistic regression analysis was used to estimate the prevalence odds ratios for asthma, adjusting for various risk factors. RESULTS: The prevalence of self-reported asthma was 1.8% (95%CI 1.6–2.0) among men and 1.9% (95%CI 1.8–2.0) among women, with higher rates in rural than in urban areas and marked geographic differences. After adjustment for known asthma risk factors, women were 1.2 times more likely 

In [12]:
'''IN-OUT Results'''
print(scores_in_out[:5])
view_files(in_out_top5)

[('./inputs/tokens/3412.tokens', -0.609987199306488), ('./inputs/tokens/1880.tokens', -0.624422013759613), ('./inputs/tokens/3067.tokens', -0.6299903988838196), ('./inputs/tokens/1046.tokens', -0.6389977931976318), ('./inputs/tokens/1525.tokens', -0.645818293094635)]
DOCUMENT NUMBER:  MED-5171
The objective of this study was to determine the prevalence of enterohemorrhagic Escherichia coli (EHEC), E. coli O157, Salmonella, and Listeria monocytogenes in retail food samples from Seattle, Wash. A total of 2,050 samples of ground beef (1,750 samples), mushrooms (100 samples), and sprouts (200 samples) were collected over a 12-month period and analyzed for the presence of these pathogens. PCR assays, followed by culture confirmation were used to determine the presence or absence of each organism. Of the 1,750 ground beef samples analyzed, 61 (3.5%) were positive for EHEC, and 20 (1.1%) of these were positive for E. coli O157. Salmonella was present in 67 (3.8%) of the 1,750 ground beef samp